## Worldometer Site Web Scrapping

---

In [1]:
# Import libraries

import os
import re
import glob
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
# Import dataset

link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# Find table

thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [4]:
# Get the table contents

head_rows = []
body_rows = []

for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
print(head_rows)

[['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl']]


In [5]:
# Save content in dataframe

df_wm = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

df_wm.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
217,210,Solomon Islands,17,,,,5,,12,,24,,"4,500","6,478","694,656",Australia/Oceania,"40,862",,154
218,211,Saint Pierre Miquelon,14,,,,14,,0,,"2,422",,"2,946","509,600","5,781",North America,413,,2
219,212,Montserrat,13,,1,,12,,0,,"2,604",200,626,"125,376","4,993",North America,384,"4,993",8
220,213,Western Sahara,10,,1,,8,,1,,17,2,,,"604,115",Africa,"60,412","604,115",
221,214,Anguilla,10,,,,10,,0,,664,,"4,043","268,353","15,066",North America,"1,507",,4


In [6]:
# Drop unwanted rows & columns

df_wm = df_wm.iloc[8:, :-3].reset_index(drop=True)
df_wm = df_wm.drop('#', axis=1)
df_wm.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"18,061,680","+173,327","323,342","+2,497","10,527,138","+132,852","7,211,200","28,002","54,417",974,"232,425,098","700,260","331,912,730",North America
1,India,"10,031,659","+26,834","145,513",+342,"9,579,681","+29,758","306,465","8,944","7,236",105,"160,090,514","115,477","1,386,345,438",Asia
2,Brazil,"7,213,155","+49,243","186,356",+669,"6,222,764","+24,579","804,035","8,318","33,822",874,"25,700,000","120,506","213,267,162",South America
3,Russia,"2,819,429","+28,209","50,347",+585,"2,254,742","+26,109","514,340","2,300","19,316",345,"84,904,111","581,679","145,963,926",Europe
4,France,"2,460,555","+17,565","60,418",+189,"183,571",+915,"2,216,566","2,727","37,657",925,"30,346,777","464,434","65,341,427",Europe


In [7]:
# Rename & rearrange columns
df_wm.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_wm = df_wm[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,912,730","18,061,680","+173,327","323,342","+2,497","10,527,138","+132,852","7,211,200","28,002","54,417",974,"232,425,098","700,260"
1,India,Asia,"1,386,345,438","10,031,659","+26,834","145,513",+342,"9,579,681","+29,758","306,465","8,944","7,236",105,"160,090,514","115,477"
2,Brazil,South America,"213,267,162","7,213,155","+49,243","186,356",+669,"6,222,764","+24,579","804,035","8,318","33,822",874,"25,700,000","120,506"
3,Russia,Europe,"145,963,926","2,819,429","+28,209","50,347",+585,"2,254,742","+26,109","514,340","2,300","19,316",345,"84,904,111","581,679"
4,France,Europe,"65,341,427","2,460,555","+17,565","60,418",+189,"183,571",+915,"2,216,566","2,727","37,657",925,"30,346,777","464,434"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# Addding 'WHO Region' & list of countries with missing "WHO Region"

df_wm['WHO Region'] = df_wm['Country/Region'].map(who_region)
df_wm[df_wm['WHO Region'].isna()]['Country/Region'].unique()

array(['French Polynesia', 'French Guiana', 'Réunion', 'Guadeloupe',
       'Mayotte', 'Martinique', 'Aruba', 'CAR', 'Curaçao',
       'Channel Islands', 'Sint Maarten', 'Gibraltar', 'Saint Martin',
       'Turks and Caicos', 'Diamond Princess', 'Faeroe Islands',
       'Isle of Man', 'Cayman Islands', 'Caribbean Netherlands',
       'St. Barth', 'Brunei ', 'St. Vincent Grenadines',
       'British Virgin Islands', 'Macao', 'New Caledonia',
       'Falkland Islands', 'Saint Pierre Miquelon', 'Montserrat',
       'Anguilla'], dtype=object)

In [10]:
# Data corrections

for col in df_wm.columns[2:]:
    # replace comma with empty string
    df_wm[col] = df_wm[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_wm[col] = df_wm[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_wm = df_wm.replace('', np.nan)

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331912730,18061680,173327,323342,2497,10527138,132852,7211200,28002,54417,974,232425098,700260,Americas
1,India,Asia,1386345438,10031659,26834,145513,342,9579681,29758,306465,8944,7236,105,160090514,115477,South-EastAsia
2,Brazil,South America,213267162,7213155,49243,186356,669,6222764,24579,804035,8318,33822,874,25700000,120506,Americas
3,Russia,Europe,145963926,2819429,28209,50347,585,2254742,26109,514340,2300,19316,345,84904111,581679,Europe
4,France,Europe,65341427,2460555,17565,60418,189,183571,915,2216566,2727,37657,925,30346777,464434,Europe


In [11]:
# Save as "csv" file

df_wm.to_csv('worldometer_data.csv', index=False)